# PlayWrightブラウザツールキット

このツールキットは、ブラウザと対話するために使用されます。他のツール（Requestsツールなど）は静的なサイトには適していますが、ブラウザツールキットではエージェントがWebをナビゲートし、動的にレンダリングされたサイトと対話できます。ブラウザツールキットに含まれるいくつかのツールは以下の通りです。

- NavigateTool（navigate_browser） - URLに移動
- NavigateBackTool（previous_page） - 要素が表示されるのを待つ
- ClickTool（click_element） - 要素をクリック（セレクタで指定）
- ExtractTextTool（extract_text） - 現在のWebページからテキストを抽出するためにBeautiful Soupを使用
- ExtractHyperlinksTool（extract_hyperlinks） - 現在のWebページからハイパーリンクを抽出するためにBeautiful Soupを使用
- GetElementsTool（get_elements） - CSSセレクタで要素を選択
- CurrentPageTool（current_page） - 現在のページのURLを取得


In [ ]:
# !pip install playwright > /dev/null
# !pip install  lxml

# If this is your first time using playwright, you'll have to install a browser executable.
# Running `playwright install` by default installs a chromium browser executable.
# playwright install

In [ ]:
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import (
    create_async_playwright_browser,
    create_sync_playwright_browser,# A synchronous browser is available, though it isn't compatible with jupyter.
)

In [ ]:
# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio
nest_asyncio.apply()

## Instantiating a Browser Toolkit

It's always recommended to instantiate using the `from_browser` method so that the 

In [ ]:
async_browser = create_async_playwright_browser()
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()
tools

In [ ]:
tools_by_name = {tool.name: tool for tool in tools}
navigate_tool = tools_by_name["navigate_browser"]
get_elements_tool = tools_by_name["get_elements"]

In [ ]:
await navigate_tool.arun({"url": "https://web.archive.org/web/20230428131116/https://www.cnn.com/world"})

In [ ]:
# The browser is shared across tools, so the agent can interact in a stateful manner
await get_elements_tool.arun({"selector": ".container__headline", "attributes": ["innerText"]})

In [ ]:
# If the agent wants to remember the current webpage, it can use the `current_webpage` tool
await tools_by_name['current_webpage'].arun({})

## Use within an Agent

Several of the browser tools are `StructuredTool`'s, meaning they expect multiple arguments. These aren't compatible (out of the box) with agents older than the `STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION`

In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatAnthropic

llm = ChatAnthropic(temperature=0) # or any other LLM, e.g., ChatOpenAI(), OpenAI()

agent_chain = initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
result = await agent_chain.arun("What are the headers on langchain.com?")
print(result)